In [ ]:
#  Copyright 2017-2021 Reveal Energy Services, Inc
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This file is part of Orchid and related technologies.
#

# Example: Plotting well monitor curves

This notebook illustrates using the Orchid* Python API and the pandas package to
plot well monitor curves (time series).

(*Orchid is a mark of Reveal Energy Services, Inc)

## 0.5 Import packages

The only import needed for the Python API is `orchid` itself.

In [ ]:
import orchid

The remaining imports are standard python packages to support the analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# The following import is included for its "side-effects" of an improved color schemes and
# plot styles. (See the "Tip" in section 9.2 of "Python for Data Analysis" for details.)
import seaborn as sns

## 1.0 Load the .ifrac project

The following code simply captures the configured location of the Orchid training data. It is not needed to
use the Orchid Python API itself, but it is used in this example to load well-known data.

In [ ]:
orchid_training_data_path = orchid.training_data_path()

In [ ]:
project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'Project_frankNstein_Permian_UTM13_FEET.ifrac')))

In [ ]:
### 1.1 Get the monitor curves (be patient)

In [ ]:
monitor_curves = [curve for curve in project.monitor_curves()]
monitor_curve_names = [curve.display_name for curve in monitor_curves]
time_series = [curve.time_series() for curve in monitor_curves]

### 1.2 Plot the curves in a 4x4 array

Define helper functions

- Calculate the curve physical quantity
- Plot a single series

In [ ]:
def plot_monitor_curve(using_series, on_axes, using_color, physical_quantity,
                       unit_abbreviation, name):
    """
    Plot the specified time series using the supplied details

    Args:
        using_series: The series to plot.
        on_axes: The axes on which to plot the series.
        using_color: The color for the plot.
        physical_quantity: The physical quantity of the series.
        unit_abbreviation: The abbreviation of the unit_abbreviation of the plotted samples.
        name: The name of the curve.
    """
    using_series.plot(ax=on_axes, color=using_color)
    on_axes.set_ylabel(f'{str(physical_quantity).title()} ({unit_abbreviation})')
    on_axes.title.set_text(name)
    x_tick_labels = on_axes.get_xticklabels()
    plt.setp(x_tick_labels, rotation=30)

In [ ]:
default_well_colors = ['#%02x%02x%02x' % tuple(map(lambda c: int(255 * c), (r, g, b)))
                       for (r, g, b) in project.default_well_colors()]
curves_shape = (2, 2)
max_curve_count = curves_shape[0] * curves_shape[1]
figure, axes = plt.subplots(*curves_shape, figsize=(10, 14), dpi=200)
curves = np.reshape(monitor_curves[:max_curve_count], curves_shape)
series_to_plot = np.reshape(time_series[:max_curve_count], curves_shape)
names_to_display = np.reshape(monitor_curve_names[:max_curve_count], curves_shape)
colors_to_use = np.reshape(default_well_colors[:max_curve_count], curves_shape)
for i in range(len(axes)):
    for j in range(len(axes[0])):
        curve = curves[i, j]
        series = series_to_plot[i, j]
        ax = axes[i, j]
        color = colors_to_use[i, j]
        quantity_name = curve.sampled_quantity_name
        abbreviation = orchid.abbreviation(curve.sampled_quantity_unit())
        series_name = names_to_display[i, j]
        plot_monitor_curve(series, ax, color, quantity_name, abbreviation, series_name)

plt.show()